In [1]:
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from methods.baseline import Baseline
from methods.retrieval import Retrieval

C:\Users\Thiago\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
retrieval = Retrieval()

path = 'data/processed/eclipse'
path_buckets = 'data/normalized/eclipse/eclipse_pairs.csv'
path_train = 'data/processed/eclipse/train.txt'
path_test = 'data/processed/eclipse/test.txt'

MAX_SEQUENCE_LENGTH_T = 100 # Title
MAX_SEQUENCE_LENGTH_D = 100 # Description

# Create the instance from baseline
retrieval.baseline = Baseline(path, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

df = pd.read_csv(path_buckets)

# Load bug ids
retrieval.load_bugs(path, path_train)
# Create the buckets
retrieval.create_bucket(df)
# Read and create the test queries duplicate
retrieval.create_queries(path_test)
# Read the siamese model
retrieval.read_model(MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

Reading train data


100%|█████████████████████████████████| 212512/212512 [17:34<00:00, 201.56it/s]


Creating the buckets...


100%|████████████████████████████████| 193599/193599 [01:36<00:00, 2005.16it/s]


Creating the queries...


12859it [00:00, 16974.74it/s]


Loaded model from disk


In [4]:
retrieval.train_vectorized, retrieval.test_vectorized = [], []
# Infer vector to all train
retrieval.read_train(path_train)
retrieval.infer_vector(retrieval.train, retrieval.train_vectorized)
# Infer vector to all test
retrieval.infer_vector(retrieval.test, retrieval.test_vectorized)

In [7]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
# inpired on https://www.kaggle.com/mauricedw22/k-nearest-neighbor-recommendation-system
# Indexing all train in KNN method
X = np.array(retrieval.train_vectorized)
nbrs = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(X)

In [14]:
# Next we find k nearest neighbor for each point in object X.
distances, indices = nbrs.kneighbors(X)

In [38]:
# Creating a hash from the train to index the clusters indices
bug_set_cluster = {}
index = 0
for row in tqdm(retrieval.train):
    dup_a_id, dup_b_id = row
    # if dup_a_id not in bug_set or dup_b_id not in bug_set: continue
    bug_set_cluster[indices[index][:1][0]] = dup_a_id
    bug_set_cluster[indices[index+1][:1][0]] = dup_b_id
    index += 2

100%|█████████████████████████████████| 70287/70287 [00:04<00:00, 17031.28it/s]


In [39]:
X_test = [retrieval.test_vectorized[1]]
distances_test, indices_test = nbrs.kneighbors(X_test)

In [43]:
bug_set_cluster[indices_test[0][:1][0]]

101443

In [ ]:
# Recommend neighborhood instances from test sample
X_test = retrieval.test_vectorized
distances_test, indices_test = nbrs.kneighbors(X_test)
# Generating the rank result